In [5]:
# import programs
import csv
import re
import os

# Removing Singlets from the data matrix

In [15]:

def process_otu_table(input_csv_path, output_csv_path, log_csv_path, sum_threshold=2):
    """
    Parses a CSV file, sums columns matching a specific pattern (e.g., A01, B12),
    and filters rows based on this sum.

    Args:
        input_csv_path (str): Path to the input CSV file.
        output_csv_path (str): Path to write the filtered CSV data.
        log_csv_path (str): Path to write the removed rows.
        sum_threshold (int): Rows with a sum of counts less than this threshold will be removed.
                             Default is 2, so sums of 0 or 1 will be removed.
    """
    try:
        with open(input_csv_path, 'r', newline='') as infile, \
             open(output_csv_path, 'w', newline='') as outfile, \
             open(log_csv_path, 'w', newline='') as logfile:

            csv_reader = csv.reader(infile)
            csv_writer = csv.writer(outfile)
            log_writer = csv.writer(logfile)

            # Read the header
            header = next(csv_reader, None)
            if not header:
                print(f"Warning: Input file '{input_csv_path}' is empty or has no header.")
                return

            # Write headers to output files
            csv_writer.writerow(header)
            log_writer.writerow(header)

            # Identify columns to sum based on pattern:
            # A capital letter followed by exactly two digits (e.g., A01, B12, Z99)
            column_pattern = re.compile(r'^[A-Z]\d{2}$')
            sum_column_indices = []
            for i, col_name in enumerate(header):
                if column_pattern.match(col_name.strip()): # .strip() to handle potential whitespace
                    sum_column_indices.append(i)

            if not sum_column_indices:
                print(f"Warning: No columns matching the pattern '[A-Z]\\d{{2}}' (e.g., A01, B12) found in the header of '{input_csv_path}'.")
                print("Please ensure your column headers for summing are like 'A01', 'S34', etc.")
            else:
                print(f"Identified columns for summing: {[header[i] for i in sum_column_indices]}")

            rows_processed = 0
            rows_kept = 0
            rows_removed = 0

            # Process each data row
            for i, row in enumerate(csv_reader):
                rows_processed += 1
                current_sum = 0
                try:
                    for col_idx in sum_column_indices:
                        if col_idx < len(row):
                            value_str = row[col_idx].strip()
                            if value_str:
                                try:
                                    current_sum += int(float(value_str))
                                except ValueError:
                                    print(f"Warning: Could not convert '{row[col_idx]}' to an integer in row {i+2} (data row {i+1}), column '{header[col_idx]}'. Treating as 0 for sum.")
                        else:
                            print(f"Warning: Row {i+2} (data row {i+1}) is shorter than header. Missing data for expected sum column '{header[col_idx]}'.")
                
                except Exception as e:
                    print(f"Error processing data in row {i+2} (data row {i+1}): {row}. Error: {e}")

                if current_sum < sum_threshold:
                    log_writer.writerow(row)
                    rows_removed += 1
                else:
                    csv_writer.writerow(row)
                    rows_kept += 1
            
            print(f"\nProcessing complete for '{input_csv_path}':")
            print(f"  Total data rows processed (excluding header): {rows_processed}")
            print(f"  Rows kept and written to '{output_csv_path}': {rows_kept}")
            print(f"  Rows removed and written to '{log_csv_path}': {rows_removed}")

    except FileNotFoundError:
        print(f"Error: Input file '{input_csv_path}' not found. Please check the file path.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


# --- Main execution point if running as a standalone Python script ---
if __name__ == "__main__":
    # Name of the input CSV file uploaded by the user
    input_filename_script = "otu_table_uclust_with_updated_taxonomy_05172025_78bats.csv"
    
    # Determine the base name for output files
    base_name_script = os.path.splitext(input_filename_script)[0]
    
    # Define output file paths
    output_filename_script = f"{base_name_script}_singletsRemoved_78bats.csv"
    log_filename_script = f"{base_name_script}_removed_rows.log_78bats.csv"

    print(f"Starting OTU table processing for: {input_filename_script} (as standalone script)")
    print(f"Filtered output will be saved to: {output_filename_script}")
    print(f"Removed rows will be logged to: {log_filename_script}")
    
    process_otu_table(input_filename_script, output_filename_script, log_filename_script, sum_threshold=2)
    
    print("\nScript finished.")


Starting OTU table processing for: otu_table_uclust_with_updated_taxonomy_05172025_78bats.csv (as standalone script)
Filtered output will be saved to: otu_table_uclust_with_updated_taxonomy_05172025_78bats_singletsRemoved_78bats.csv
Removed rows will be logged to: otu_table_uclust_with_updated_taxonomy_05172025_78bats_removed_rows.log_78bats.csv
Identified columns for summing: ['A01', 'A02', 'A03', 'A04', 'A05', 'A08', 'A09', 'A10', 'A12', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B08', 'B09', 'B10', 'B12', 'C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08', 'C09', 'C10', 'C12', 'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D09', 'D10', 'D11', 'D12', 'E01', 'E02', 'E03', 'E04', 'E06', 'E07', 'E08', 'E09', 'E10', 'E11', 'E12', 'F01', 'F02', 'F03', 'F04', 'F06', 'F07', 'F08', 'F09', 'F10', 'F11', 'G01', 'G02', 'G03', 'G04', 'G07', 'G08', 'G09', 'G10', 'G11', 'H01', 'H02', 'H03', 'H04', 'H07', 'H08', 'H09', 'H11']

Processing complete for 'otu_table_uclust_with_updated_taxonomy_0

# Filtering out Metazoa and No Classification Taxa

In [20]:
import csv
import os

def filter_csv_by_domain(input_csv_path, filtered_csv_path, log_csv_path, domain_column_name="Domain", values_to_remove=None):
    """
    Filters a CSV file by removing rows where the specified domain column contains certain values.

    Args:
        input_csv_path (str): Path to the input CSV file.
        filtered_csv_path (str): Path to write the filtered CSV data (rows kept).
        log_csv_path (str): Path to write the log of removed rows.
        domain_column_name (str): The name of the column to check for domain values.
        values_to_remove (list): A list of string values. If the domain column matches
                                 any of these values (case-insensitive), the row is removed.
    """
    if values_to_remove is None:
        values_to_remove = ["metazoa", "no classification"] # Default values, made lowercase for case-insensitive comparison
    else:
        # Ensure comparison values are lowercase for case-insensitivity
        values_to_remove = [str(val).lower() for val in values_to_remove]

    rows_processed = 0
    rows_kept = 0
    rows_removed = 0
    domain_col_idx = -1

    try:
        with open(input_csv_path, 'r', newline='', encoding='utf-8') as infile, \
             open(filtered_csv_path, 'w', newline='', encoding='utf-8') as outfile_filtered, \
             open(log_csv_path, 'w', newline='', encoding='utf-8') as outfile_log:

            csv_reader = csv.reader(infile)
            csv_writer_filtered = csv.writer(outfile_filtered)
            csv_writer_log = csv.writer(outfile_log)

            # Read and write the header
            header = next(csv_reader, None)
            if not header:
                print(f"Error: Input file '{input_csv_path}' is empty or has no header.")
                return

            csv_writer_filtered.writerow(header)
            csv_writer_log.writerow(header)

            # Find the index of the domain column
            try:
                domain_col_idx = header.index(domain_column_name)
            except ValueError:
                print(f"Error: Column '{domain_column_name}' not found in the header of '{input_csv_path}'.")
                print(f"Available columns: {header}")
                # Write all rows to filtered output if domain column is not found, as filtering cannot be applied
                for row in csv_reader:
                    csv_writer_filtered.writerow(row)
                    rows_processed +=1
                    rows_kept +=1
                print(f"\nSince '{domain_column_name}' column was not found, all {rows_processed} data rows were written to '{filtered_csv_path}'.")
                return

            # Process each data row
            for row in csv_reader:
                rows_processed += 1
                try:
                    domain_value = row[domain_col_idx].strip().lower() # Get value and make it lowercase
                    
                    # Check if the domain value is one of the values to remove
                    # This also handles cases where the domain might be part of a larger string,
                    # e.g. "Metazoa (kingdom)" would match "metazoa" if "metazoa" is in values_to_remove
                    # For exact match, use: if domain_value in values_to_remove:
                    should_remove = False
                    for removal_term in values_to_remove:
                        if removal_term in domain_value: # Check if the term is a substring
                            should_remove = True
                            break
                    
                    if should_remove:
                        csv_writer_log.writerow(row)
                        rows_removed += 1
                    else:
                        csv_writer_filtered.writerow(row)
                        rows_kept += 1
                except IndexError:
                    # This might happen if a row has fewer columns than the header
                    print(f"Warning: Row {rows_processed + 1} has an unexpected number of columns. Skipping.")
                    # Optionally, write this problematic row to the log or a separate error file
                    csv_writer_log.writerow(row + ["Error: Malformed row"]) 
                    rows_removed +=1


            print(f"\nProcessing complete for '{input_csv_path}':")
            print(f"  Total data rows processed (excluding header): {rows_processed}")
            print(f"  Rows kept and written to '{filtered_csv_path}': {rows_kept}")
            print(f"  Rows removed and written to '{log_csv_path}': {rows_removed}")

    except FileNotFoundError:
        print(f"Error: Input file '{input_csv_path}' not found. Please check the file path.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# --- Main execution point of the script ---
if __name__ == "__main__":
    # Name of the input CSV file uploaded by the user
    input_filename = "otu_table_uclust_with_updated_taxonomy_05172025_78bats_singletsRemoved_78bats.csv"
    
    # Determine the base name for output files
    base_name_input = os.path.splitext(input_filename)[0]
    
    # Define output file paths
    filtered_output_filename = f"{base_name_input}_filtered_domains.csv"
    log_filename = f"{base_name_input}_removed_domain_rows.log.csv"

    # Define the specific values in the "Domain" column that should trigger row removal
    # The comparison will be case-insensitive and will check if these terms are *contained*
    # within the domain string. For exact matches, the logic inside the function would need adjustment.
    domains_to_remove = ["Metazoa", "No Classification"] # The function will handle case-insensitivity

    print(f"Starting CSV filtering for: {input_filename}")
    print(f"Rows where 'Domain' contains {domains_to_remove} (case-insensitive) will be removed.")
    print(f"Filtered data will be saved to: {filtered_output_filename}")
    print(f"Removed rows will be logged to: {log_filename}")
    
    filter_csv_by_domain(input_filename, 
                         filtered_output_filename, 
                         log_filename, 
                         domain_column_name="Domain", 
                         values_to_remove=domains_to_remove)
    
    print("\nScript finished.")


Starting CSV filtering for: otu_table_uclust_with_updated_taxonomy_05172025_78bats_singletsRemoved_78bats.csv
Rows where 'Domain' contains ['Metazoa', 'No Classification'] (case-insensitive) will be removed.
Filtered data will be saved to: otu_table_uclust_with_updated_taxonomy_05172025_78bats_singletsRemoved_78bats_filtered_domains.csv
Removed rows will be logged to: otu_table_uclust_with_updated_taxonomy_05172025_78bats_singletsRemoved_78bats_removed_domain_rows.log.csv

Processing complete for 'otu_table_uclust_with_updated_taxonomy_05172025_78bats_singletsRemoved_78bats.csv':
  Total data rows processed (excluding header): 1939
  Rows kept and written to 'otu_table_uclust_with_updated_taxonomy_05172025_78bats_singletsRemoved_78bats_filtered_domains.csv': 1511
  Rows removed and written to 'otu_table_uclust_with_updated_taxonomy_05172025_78bats_singletsRemoved_78bats_removed_domain_rows.log.csv': 428

Script finished.


# Combining like Genera together 
- combines all like genera together in one row
- leaves the blank columns
- new OTU_ID reads like sum_denovo####, denovo####, etc.)

In [22]:
import pandas as pd
import os
import re

def combine_rows_by_genus(input_csv_path, output_csv_path, log_file_path,
                          genus_col_name="Genus", otu_id_col_identifier=None):
    """
    Combines rows in a CSV file based on identical entries in the 'Genus' column.
    Numeric sample columns are summed. OTU IDs are aggregated.
    Rows with blank Genus entries are kept as is and appended to the output.

    Args:
        input_csv_path (str): Path to the input CSV file.
        output_csv_path (str): Path to save the combined CSV data.
        log_file_path (str): Path to save the log of combined OTU IDs.
        genus_col_name (str): Name of the column containing Genus information.
        otu_id_col_identifier (str, optional): Name of the OTU ID column. 
                                             If None, assumes the first column.
    """
    try:
        df = pd.read_csv(input_csv_path)
        print(f"Successfully loaded '{input_csv_path}'. Shape: {df.shape}")
    except FileNotFoundError:
        print(f"Error: Input file '{input_csv_path}' not found.")
        return
    except Exception as e:
        print(f"Error loading CSV: {e}")
        return

    if df.empty:
        print("Input CSV is empty. Nothing to process.")
        return

    # Determine OTU ID column
    if otu_id_col_identifier and otu_id_col_identifier in df.columns:
        actual_otu_id_col = otu_id_col_identifier
    elif df.columns[0]: 
        actual_otu_id_col = df.columns[0]
        print(f"Assuming '{actual_otu_id_col}' is the OTU ID column (first column).")
    else:
        print("Error: Could not determine OTU ID column.")
        return

    if genus_col_name not in df.columns:
        print(f"Error: Genus column '{genus_col_name}' not found in the CSV.")
        print(f"Available columns: {df.columns.tolist()}")
        return

    # Identify numeric sample columns (e.g., A01, B12)
    sample_col_pattern = re.compile(r'^[A-Z]\d{2}$')
    numeric_sample_cols = [col for col in df.columns if sample_col_pattern.match(col)]
    if not numeric_sample_cols:
        print("Warning: No numeric sample columns matching pattern '[A-Z]\\d{2}' found. Summation might not work as expected.")
    else:
        print(f"Identified numeric sample columns for summation: {numeric_sample_cols}")
        for col in numeric_sample_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

    standard_taxonomy_cols = ["Domain", "Phylum", "Class", "Order", "Family", "Species"]
    other_taxonomy_cols = [col for col in standard_taxonomy_cols if col in df.columns and col != genus_col_name]
    print(f"Identified other taxonomy columns to carry over: {other_taxonomy_cols}")

    # Prepare Genus column and identify blank entries
    df[genus_col_name] = df[genus_col_name].astype(str).str.strip() # Ensure string type and strip whitespace
    blank_genus_indicators = ['', 'nan', 'none', 'na', '<na>'] # Made lowercase for consistent check
    
    # Separate rows with blank Genus from those to be combined
    is_blank_genus_mask = df[genus_col_name].str.lower().isin(blank_genus_indicators) | df[genus_col_name].isnull()
    df_blank_genus = df[is_blank_genus_mask].copy()
    df_to_combine = df[~is_blank_genus_mask].copy()

    print(f"{len(df_blank_genus)} rows with blank Genus will be kept as is and appended.")
    print(f"{len(df_to_combine)} rows with non-blank Genus will be processed for combination.")

    df_combined_aggregated = pd.DataFrame() # Initialize empty DataFrame for aggregated results

    if not df_to_combine.empty:
        agg_funcs = {}
        def aggregate_otu_ids(x):
            ids = ", ".join(sorted(list(set(x.astype(str)))))
            return f"sum of {ids}" if ids else "N/A"
        
        agg_funcs[actual_otu_id_col] = aggregate_otu_ids

        for col in numeric_sample_cols:
            agg_funcs[col] = 'sum'
        for col in other_taxonomy_cols:
            agg_funcs[col] = 'first'

        print(f"\nStarting aggregation for non-blank Genus entries by '{genus_col_name}'...")
        df_to_combine[actual_otu_id_col] = df_to_combine[actual_otu_id_col].astype(str)

        try:
            df_combined_aggregated = df_to_combine.groupby(genus_col_name, as_index=False).agg(agg_funcs)
            print("Aggregation for non-blank Genus entries complete.")
        except Exception as e:
            print(f"Error during aggregation: {e}")
            print("Aggregation functions defined:", agg_funcs)
            print("Columns in df_to_combine:", df_to_combine.columns.tolist())
            # If aggregation fails, we might still want to output the blank genus rows
            # For now, we'll let it proceed to concat, df_combined_aggregated will be empty.
    else:
        print("No non-blank Genus entries to aggregate.")

    # Concatenate aggregated data with rows that had blank Genus
    # Ensure columns are aligned; df_blank_genus has original columns.
    # df_combined_aggregated might have fewer columns if some taxonomy levels were all NaN for a genus.
    # We need to make sure the final output has all original columns in the original order.
    
    final_df_list = []
    if not df_combined_aggregated.empty:
        final_df_list.append(df_combined_aggregated)
    if not df_blank_genus.empty:
        # Ensure df_blank_genus has the same columns as the original df for proper concatenation order
        # If df_combined_aggregated is empty, the output columns should match original df
        # If df_combined_aggregated is not empty, its columns define the primary structure for aggregated part
        final_df_list.append(df_blank_genus)

    if not final_df_list:
        print("No data to write to output file (neither aggregated nor blank genus rows).")
        # Create empty file with headers if original df was not empty
        if not df.empty:
            pd.DataFrame(columns=df.columns).to_csv(output_csv_path, index=False)
            with open(log_file_path, 'w', encoding='utf-8') as log_f:
                log_f.write("Genus,Combined_OTU_IDs_Raw\n") # Log will be empty if no combinations
        return

    df_final_output = pd.concat(final_df_list, ignore_index=True)

    # Reorder columns to match the original input as much as possible
    # Use original df's column order as the reference
    original_columns = df.columns.tolist()
    # Filter to columns present in the final output, maintaining original order
    final_ordered_columns = [col for col in original_columns if col in df_final_output.columns]
    # Add any new columns created during aggregation (shouldn't be any if logic is right)
    # or columns that might have been only in one part of the concat (e.g. only in blank_genus part)
    for col in df_final_output.columns:
        if col not in final_ordered_columns:
            final_ordered_columns.append(col)
            
    df_final_output = df_final_output[final_ordered_columns]


    try:
        df_final_output.to_csv(output_csv_path, index=False)
        print(f"\nCombined data (including rows with blank Genus) saved to '{output_csv_path}'. Shape: {df_final_output.shape}")
    except Exception as e:
        print(f"Error saving final combined CSV: {e}")

    # Write the log file for combined OTUs (only for non-blank Genus entries that were combined)
    try:
        with open(log_file_path, 'w', encoding='utf-8') as log_f:
            log_f.write("Genus,Combined_OTU_IDs_Raw\n")
            if not df_to_combine.empty: # Log only if there was data to combine
                for genus, group_df in df_to_combine.groupby(genus_col_name):
                    otu_ids_for_log = ", ".join(sorted(list(set(group_df[actual_otu_id_col].astype(str)))))
                    log_f.write(f"\"{genus}\",\"{otu_ids_for_log}\"\n")
        print(f"Log of combined OTU IDs (for non-blank Genus) saved to '{log_file_path}'")
    except Exception as e:
        print(f"Error writing log file: {e}")


if __name__ == "__main__":
    input_filename = "otu_table_uclust_with_updated_taxonomy_05172025_78bats_singletsRemoved_78bats_filtered_domains.csv"
    
    base_name_input = os.path.splitext(input_filename)[0]
    
    output_filename = f"{base_name_input}_combined_by_genus_keep_blanks.csv" # Updated output name
    log_filename = f"{base_name_input}_genus_combination_log_keep_blanks.txt" # Updated log name

    print(f"Starting CSV processing for: {input_filename}")
    print(f"Rows with blank Genus will be kept as is at the end of the file.")
    print(f"Output will be saved to: {output_filename}")
    print(f"Combination log (for non-blank Genus) will be saved to: {log_filename}")

    combine_rows_by_genus(input_filename, 
                          output_filename, 
                          log_filename,
                          genus_col_name="Genus",
                          otu_id_col_identifier=None) 
    
    print("\nScript finished.")


Starting CSV processing for: otu_table_uclust_with_updated_taxonomy_05172025_78bats_singletsRemoved_78bats_filtered_domains.csv
Rows with blank Genus will be kept as is at the end of the file.
Output will be saved to: otu_table_uclust_with_updated_taxonomy_05172025_78bats_singletsRemoved_78bats_filtered_domains_combined_by_genus_keep_blanks.csv
Combination log (for non-blank Genus) will be saved to: otu_table_uclust_with_updated_taxonomy_05172025_78bats_singletsRemoved_78bats_filtered_domains_genus_combination_log_keep_blanks.txt
Successfully loaded 'otu_table_uclust_with_updated_taxonomy_05172025_78bats_singletsRemoved_78bats_filtered_domains.csv'. Shape: (1511, 89)
Assuming 'OTU_ID' is the OTU ID column (first column).
Identified numeric sample columns for summation: ['A01', 'A02', 'A03', 'A04', 'A05', 'A08', 'A09', 'A10', 'A12', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B08', 'B09', 'B10', 'B12', 'C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08', 'C09', 'C10', 'C12', 'D01', 'D